In [2]:
! git clone https://github.com/gyyang/neurogym.git
%cd neurogym/
! pip install -e .

Cloning into 'neurogym'...
remote: Enumerating objects: 11100, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 11100 (delta 132), reused 234 (delta 115), pack-reused 10827
Receiving objects: 100% (11100/11100), 8.20 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (8318/8318), done.
/home/maria/Neuromatch2024/neurogym
Obtaining file:///home/maria/Neuromatch2024/neurogym
  Preparing metadata (setup.py) ... done
  Running setup.py develop for neurogym

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install gymnasium


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [24]:
import gym
import neurogym as ngym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import numpy as np


# Create a custom multi-task environment
import gym
import neurogym as ngym
import numpy as np

class MultiTaskEnv(gym.Env):
    def __init__(self, task1_id, task2_id, switch_interval):
        super(MultiTaskEnv, self).__init__()
        self.task1 = ngym.make(task1_id)
        self.task2 = ngym.make(task2_id)
        self.switch_interval = switch_interval
        self.current_step = 0

        # Concatenate observation spaces
        self.observation_space = gym.spaces.Box(
            low=np.concatenate([self.task1.observation_space.low, self.task2.observation_space.low]),
            high=np.concatenate([self.task1.observation_space.high, self.task2.observation_space.high]),
            dtype=self.task1.observation_space.dtype
        )

        # Create a joint action space
        print('boom', self.task1.action_space, self.task2.action_space)
        self.action_space = gym.spaces.Tuple((self.task1.action_space, self.task2.action_space))

        self.task1.reset()
        self.task2.reset()

    def reset(self):
        self.current_step = 0
        obs1 = self.task1.reset()
        obs2 = self.task2.reset()
        concatenated_obs = np.concatenate([obs1, obs2])
        return concatenated_obs

    def step(self, action):
        action1, action2 = action
        print('boom',action1,action2)
        self.current_step += 1

        # Step task1 with action1 (Discrete)
        obs1, reward1, done1, info1 = self.task1.step(action1.item())  # Convert action1 tensor to scalar
        # Step task2 with action2 (Box)
        obs2, reward2, done2, info2 = self.task2.step(action2.tolist())  # Convert action2 tensor to list

        concatenated_obs = np.concatenate([obs1, obs2])
        reward = reward1 + reward2
        done = done1 or done2
        info = {**info1, **info2}

        if done:
            obs1 = self.task1.reset()
            obs2 = self.task2.reset()
            concatenated_obs = np.concatenate([obs1, obs2])

        return concatenated_obs, reward, done, info



# Define the custom MLP policy
import torch
import torch.nn as nn
from torch.distributions import Categorical

import torch
import torch.nn as nn
from torch.distributions import Categorical, Normal

class MLPPolicy(nn.Module):
    def __init__(self, obs_space, act_space1, act_space2, hidden_sizes=[128, 128], activation=nn.ReLU):
        super(MLPPolicy, self).__init__()
        self.obs_space = obs_space
        self.act_space1 = act_space1
        self.act_space2 = act_space2

        layers = []
        input_size = obs_space.shape[0]
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(activation())
            input_size = hidden_size

        self.shared_net = nn.Sequential(*layers)

        # Policy head for Discrete action space
        self.policy_net1 = nn.Linear(hidden_sizes[-1], act_space1.n)

        # Policy head for Box action space (mean and log_std)
        self.policy_net2_mean = nn.Linear(hidden_sizes[-1], act_space2.shape[0])
        self.policy_net2_log_std = nn.Linear(hidden_sizes[-1], act_space2.shape[0])

        # Value network
        self.value_net = nn.Sequential(
            nn.Linear(obs_space.shape[0], hidden_sizes[0]),
            activation(),
            nn.Linear(hidden_sizes[0], 1)
        )

    def forward(self, x):
        shared_features = self.shared_net(x)
        policy_logits1 = self.policy_net1(shared_features)
        policy_mean2 = self.policy_net2_mean(shared_features)
        policy_log_std2 = self.policy_net2_log_std(shared_features)
        value = self.value_net(x)
        return policy_logits1, policy_mean2, policy_log_std2, value

    def act(self, x):
        policy_logits1, policy_mean2, policy_log_std2, value = self.forward(x)
        probs1 = torch.softmax(policy_logits1, dim=-1)
        dist1 = Categorical(probs1)
        action1 = dist1.sample()
        log_prob1 = dist1.log_prob(action1)

        std2 = policy_log_std2.exp()
        dist2 = Normal(policy_mean2, std2)
        action2 = dist2.sample()
        log_prob2 = dist2.log_prob(action2).sum(dim=-1)  # Sum log probs for multi-dimensional actions

        return (action1, action2), (log_prob1, log_prob2), value




# Define the PPO agent
class PPOAgent:
    def __init__(self, policy, lr=3e-4, gamma=0.99, k_epochs=4, eps_clip=0.2, device='cuda'):
        self.policy = policy.to(device)
        self.optimizer = optim.Adam(policy.parameters(), lr=lr)
        self.gamma = gamma
        self.k_epochs = k_epochs
        self.eps_clip = eps_clip
        self.device = device

    def compute_returns(self, rewards, masks, values, next_value):
        returns = []
        R = next_value
        for step in reversed(range(len(rewards))):
            R = rewards[step] + self.gamma * R * masks[step]
            returns.insert(0, R)
        return returns

    def update(self, memory):
        states, actions, log_probs, rewards, masks, values = memory

        states = torch.stack(states).to(self.device)
        actions = torch.tensor(actions).to(self.device)
        log_probs = torch.tensor(log_probs).to(self.device)
        values = torch.stack(values).to(self.device)

        next_value = self.policy(states[-1].unsqueeze(0))[1]
        returns = self.compute_returns(rewards, masks, values, next_value)
        returns = torch.tensor(returns).detach().to(self.device)

        old_states = states.detach()
        old_actions = actions.detach()
        old_log_probs = log_probs.detach()

        for _ in range(self.k_epochs):
            logits, state_values = self.policy(old_states)
            dist = Categorical(torch.softmax(logits, dim=-1))
            new_log_probs = dist.log_prob(old_actions)
            entropy = dist.entropy().mean()

            advantages = returns - state_values.squeeze().detach()
            ratio = torch.exp(new_log_probs - old_log_probs)
            surr1 = ratio * advantages
            surr2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            actor_loss = -torch.min(surr1, surr2).mean()
            critic_loss = nn.MSELoss()(state_values.squeeze(), returns)

            loss = 0.5 * critic_loss + actor_loss - 0.01 * entropy

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


# Training loop
def train(env, agent, num_episodes=10, batch_size=32):
    memory = deque(maxlen=batch_size)
    episode_rewards = []

    for episode in range(num_episodes):
        state = env.reset()
        #env.reset()
        state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)
        episode_reward = 0

        for t in range(1, 100):
            #env.reset()
            action, log_prob, value = agent.policy.act(state)
            #action = action.item()  # Convert action tensor to scalar
            #print(action)
            next_state, reward, done, _ = env.step(action)
            next_state = torch.FloatTensor(next_state).to(agent.device).unsqueeze(0)

            memory.append((state, action, log_prob.item(), reward, 1 - int(done), value))
            episode_reward += reward

            if len(memory) == batch_size:
                states, actions, log_probs, rewards, masks, values = zip(*memory)
                agent.update((states, actions, log_probs, rewards, masks, values))
                memory.clear()

            state = next_state
            if done:
                break

        episode_rewards.append(episode_reward)
        if (episode + 1) % 10 == 0:
            print(f"Episode {episode + 1}, Reward: {episode_reward}")

    return episode_rewards


# Main execution
task1_id = 'PerceptualDecisionMaking-v0'
task2_id = 'ReachingDelayResponse-v0'
switch_interval = 100
env = MultiTaskEnv(task1_id, task2_id, switch_interval)

obs_space = env.observation_space
act_space = env.action_space
print(act_space)

policy = MLPPolicy(obs_space, act_space[0],act_space[1])
agent = PPOAgent(policy)

episode_rewards = train(env, agent)

# Evaluate the model
state = env.reset()
state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)
for _ in range(1000):
    action, _, _ = policy.act(state)
    action = action # Convert action tensor to scalar
    #print(env)
    next_state, reward, done, _ = env.step(action)
    next_state = torch.FloatTensor(next_state).to(agent.device).unsqueeze(0)
    #env.render()
    state = next_state
    if done:
        state = env.reset()
        state = torch.FloatTensor(state).to(agent.device).unsqueeze(0)

env.close()


boom Discrete(3) Box(-1.0, [1. 2.], (2,), float32)
Tuple(Discrete(3), Box(-1.0, [1. 2.], (2,), float32))
boom tensor([0], device='cuda:0') tensor([[-0.9291, -1.5587]], device='cuda:0')


TypeError: '<' not supported between instances of 'list' and 'int'